In [1]:
import pandas as pd
from parse import parse_htmls
from util import read_place_desc, read_htmls_in, write_places_to_tsv, read_places
from service import PlaceService, SearchEngine
from index import preprocess, Index, TfIdfIndex

import requests
from tqdm import tqdm
import os
import time
import util

# Data collection
##  1.1. Get the list of places





In [ ]:
with open('most_popular_places.txt', 'w') as file:
    for i in tqdm(range(300)):
        
        url = 'https://www.atlasobscura.com/places?page='+str(i+1)+'&sort=likes_count'
        list_page = requests.get(url)
        list_soup = BeautifulSoup(list_page.text)
        list_places = [x['href'] for x in list_soup.find_all('a', {'class':"content-card content-card-place"})]
        for place in tqdm(list_places):
            file.write('https://www.atlasobscura.com' + str(place))
            file.write('\n')

## 1.2. Crawl places

In [ ]:
f= open('most_popular_places.txt', 'r')
for j, url in enumerate(f):
    
    if j %18 == 0:
        dir_path = f'page{j//18+1}'
        if not os.path.isdir(dir_path):
            os.mkdir(dir_path)
            
    response = requests.get(url.strip())
    with open(f'{dir_path}/{j+1}.html','w') as file:
        print()
        file.write(response.text)
        
    time.sleep(1)

# 1.3 Parse Pages

In [3]:
#all_htmls = read_htmls_in('./pages')

In [2]:
#%store -r htmls

In [4]:
#all_places = parse_htmls(htmls)

In [5]:
#tsv_path = write_places_to_tsv('./', all_places)

In [2]:
read_places('./places.tsv')

,id,name,tags,num_people_visited,num_people_want,desc,short_desc,nearby,address,lat,lon,editors,publication_date,appears_in,related_places,url
0,3535434637316363800,City Hall Station,"abandoned,infrastructure,subterranean,architec...",1834,8603,The first New York City subway was built and o...,A beautiful and abandoned New York subway stat...,"African Burial Ground National Monument,Tunnel...","31 Centre St, New York, New York, 10007, Unite...",40.713400,-74.004600,"Rebekah Otto,Annetta Black,Allan,charding407,w...",2010-05-08 15:45:00,30 Unexpected Places to Have a Joyful Adventur...,"Crystal Palace Subway,Moscow Metro Stations,Ro...",https://www.atlasobscura.com/places/city-hall-...
1,-3332975814412938035,Dream House,"outsider art,music,optical oddities,sacred spa...",765,6007,"When walking down Church Street in Tribeca, ke...","La Monte Young and Marian Zazeela's ""Dream Hou...","Aretha Franklin Subway Tributes,Farm.One,The '...","275 Church St, New York, New York, 10013, Unit...",40.718463,-74.004784,"seanmattison,mbison,Gray,Sofy,erosika,Blindcol...",2009-09-22 14:44:00,30 Unexpected Places to Have a Joyful Adventur...,"Callejon de Hamel,Casa Neverlandia,Casa de Azu...",https://www.atlasobscura.com/places/dream-house
2,-2064942810649273700,The Museum of Death,"museums and collections,collections,crime and ...",1761,5832,"The Museum of Death, a showcase of just that, ...",World's largest collection of serial killer ar...,"Hollywood Tower Apartments,Capitol Records Bui...","6031 Hollywood Blvd., Los Angeles, California,...",34.101747,-118.321185,"Genki,hana,Lauren Levesque,britbarbie4,technob...",2010-08-10 03:05:00,"The World's Top 100 Wonders in 2018,Los Angele...","Museum Tot Zover,Roman Urn Collection at SAMA,...",https://www.atlasobscura.com/places/the-museum...
3,-2642997787417301743,The Last Bookstore,"collections,repositories of knowledge,bookstor...",3614,5816,More than a few bibliophiles would prefer that...,This iconic L.A. bookshop is housed in an aban...,"Biddy Mason Memorial Park,Bradbury Building,An...","453 S. Spring St., Los Angeles, California, 90...",34.047738,-118.249422,"kcmick,paulruta,Avoiding Regret,JesseMan,Mark ...",2013-02-27 06:30:00,The Ultimate Guide to Wondrous Independent Boo...,"Alabama Booksmith,Book Corner,John K. King Use...",https://www.atlasobscura.com/places/last-books...
4,7201689281338630909,God&#39;s Own Junkyard,"neon,signs,art,collections",856,5812,"For years, Walthamstow was known as an unremar...",A kaleidoscopic warehouse-maze of handmade neo...,"The Ancient House,The Bremer Car,Walthamstow M...","82A Shernhall St, London, England, United King...",51.583870,-0.008269,"FrustratedFormerContributor,ryanswift,ionamcme...",2017-04-18 20:00:00,15 Places That Don't Need the Holidays to Twin...,"Ignite Sign Art Museum,Buchstabenmuseum,Lite B...",https://www.atlasobscura.com/places/gods-own-j...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,3939494857715772897,The Bruce Peninsula Grotto,"grottoes,caves,water,ecosystems",266,1044,Hidden in the cliff face of an escarpment on O...,This natural sea cave hides a small wonderland...,"Flowerpot Island,Manitou Lake,Burwash Correcti...","Bruce Trail, Tobermory, Ontario, N0H 2R0, Cana...",45.245225,-81.524306,"xenip,kostas1964ath,claresmith,chriswjohnstone...",2015-11-10 10:00:00,NaN,"Las Grutas de Tolantongo,Westcave Outdoor Disc...",https://www.atlasobscura.com/places/the-bruce-...
7196,5746829262401357004,Janis Joplin’s Hotel Room,"music,hotels,shrines,places to stay",147,1044,"October 4, 1970, Janis Joplin, one of the most...",The hotel room where Janis Joplin tragically d...,"The Magic Castle,Miniature Garden of Whimsy,Ca...","7047 Franklin Ave, Los Angeles, California, 90...",34.104387,-118.343198,"erinjean,Mike Portanova",2015-08-04 11:00:00,NaN,"Gran Hotel Bolivar,Rock It Suda,Mount Washingt...",https://www.atlasobscura.com/places/janis-jopl...
7197,3872125778913922632,Shopsin&#39;s,"shops,unique restaurants & b

In [263]:
#%store htmls

Stored 'htmls' (list)


In [5]:
ids, descriptions = read_place_desc('./places.tsv')
place_service = PlaceService()
place_service.load('./places.tsv')

# Create Conjunctive Index

In [3]:
index = Index.create_from(ids, descriptions)

# Create Search Engine

In [4]:
search_engine = SearchEngine(index, place_service)

# Run Search

In [5]:
search_engine.query('american museum')

,name,desc,url
6638,Thorne Miniature Rooms,In the depths of the Museum of the Art Institu...,https://www.atlasobscura.com/places/thorne-min...
2920,Grace Hopper&#39;s Bug,"On September 9, 1947, Harvard’s Mark II Aiken ...",https://www.atlasobscura.com/places/grace-hopp...
6306,Biosphere of Montreal,As their contribution to Montreal’s 1967 World...,https://www.atlasobscura.com/places/biosphere-...
5999,The Witch House of Salem,The Salem witchcraft trials took place between...,https://www.atlasobscura.com/places/witch-hous...
4745,Susanna Dickinson Museum,According to a plaque outside her last survivi...,https://www.atlasobscura.com/places/susanna-di...
...,...,...,...
576,Truth or Consequences,Located along the Rio Grande in middle of the ...,https://www.atlasobscura.com/places/truth-or-c...
718,"Basilica of Saint Lawrence, Asheville",The turn of the 20th century was a time of pro...,https://www.atlasobscura.com/places/basilica-o...
359,Stained Glass at Navy Pier,The very first American exhibit dedicated sole...,https://www.atlasobscura.com/places/stained-gl...
516,Abandoned Comet Diner,Just down the street from Mark Twain’s house a...,https://www.atlasobscura.com/places/abandoned-...


# Create TF-IDF Index

In [ ]:
tf_idf_index = TfIdfIndex.create_from(ids, descriptions)

# Create TF-IDF Search Engine

In [ ]:
tf_idf_search_engine = SearchEngine(tf_idf_index, place_service)

In [ ]:
tf_idf_search_engine.query_top_k("american museum", 10)